In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os
from pathlib import Path

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# from animal_shelter import AnimalShelter
from CRUD_Python_Module import MongoCRUD as AnimalShelter   # <--- uses your provided CRUD module/class

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "pikachu123"

# Connect to database via CRUD Module
db = AnimalShelter(
    username,
    password,
    host="localhost",
    port=27017,
    db_name="aac",                 # <--- typical DB name for the AAC dataset
    collection_name="animals",
    auth_source="admin"
)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image

# robust loader: look in CWD, else search subfolders; fallback to None if not found
def _load_logo_bytes(fname: str):
    p = Path(fname)
    if p.exists():
        return p.read_bytes()
    hits = list(Path.cwd().rglob(fname))
    return hits[0].read_bytes() if hits else None

_logo_bytes = _load_logo_bytes(image_filename)
encoded_image = base64.b64encode(_logo_bytes).decode() if _logo_bytes else None

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(
        encoded_image and html.Img(
            src=f'data:image/png;base64,{encoded_image}',
            style={"height": "80px"},
            title="Grazioso Salvare — Dundi V"
        )
    ),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        children=[
            html.Div([
                html.Label("Rescue Filter (select one):"),
                dcc.RadioItems(
                    id="filter-type",
                    options=[
                        {"label": "Reset (All Animals)", "value": "reset"},
                        {"label": "Water Rescue", "value": "water"},
                        {"label": "Mountain/Wilderness Rescue", "value": "wilderness"},
                        {"label": "Disaster/Individual Tracking", "value": "disaster"},
                    ],
                    value="reset",
                    inline=True
                )
            ], style={"margin":"8px 0"})
        ]

    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable="single",
                         row_selectable="single",
                         page_action="native",
                         page_current=0,
                         page_size=10,
                         style_table={"overflowX": "auto"},
                         style_cell={"textAlign": "left", "minWidth":"120px", "maxWidth":"240px", "whiteSpace":"normal"}
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


# Helper: build Mongo query for each rescue filter
def _filter_query(kind: str):
    if kind == "water":
        # Water Rescue
        return {
            "animal_type": "Dog",
            "breed": {"$in": [
                "Labrador Retriever Mix", "Labrador Retriever",
                "Chesapeake Bay Retriever", "Newfoundland"
            ]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    if kind == "wilderness":
        # Mountain/Wilderness Rescue
        return {
            "animal_type": "Dog",
            "breed": {"$in": [
                "German Shepherd", "Alaskan Malamute",
                "Old English Sheepdog", "Siberian Husky", "Rottweiler"
            ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    if kind == "disaster":
        # Disaster/Individual Tracking
        return {
            "animal_type": "Dog",
            "breed": {"$in": [
                "Doberman Pinscher", "German Shepherd",
                "Golden Retriever", "Bloodhound", "Rottweiler"
            ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    # reset / default
    return {}

    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries
#
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)
    query = _filter_query(filter_type)
    records = db.read(query)
    # make sure _id is dropped if present
    for r in records:
        r.pop("_id", None)
    return records


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame(viewData) if viewData is not None and len(viewData) else df.copy()
    if "_id" in dff.columns:
        dff = dff.drop(columns=["_id"])
    if "breed" not in dff.columns or len(dff) == 0:
        return []
    counts = dff["breed"].value_counts().reset_index()
    counts.columns = ["breed", "count"]

    return [
        dcc.Graph(
            figure=px.pie(counts, names="breed", values="count", title="Breed distribution (current filter)")
        )
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in (selected_columns or [])]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Attempt by column name first; fall back to iloc positions used in the scaffold if needed
    lat_col = "location_lat" if "location_lat" in dff.columns else None
    lon_col = "location_long" if "location_long" in dff.columns else None
    breed_col = "breed" if "breed" in dff.columns else None
    name_col  = "name"  if "name"  in dff.columns else None

    try:
        lat = float(dff.iloc[row][lat_col]) if lat_col else float(dff.iloc[row,13])
        lon = float(dff.iloc[row][lon_col]) if lon_col else float(dff.iloc[row,14])
        breed_val = dff.iloc[row][breed_col] if breed_col else dff.iloc[row,4]
        name_val  = dff.iloc[row][name_col]  if name_col  else dff.iloc[row,9]
    except Exception:
        # If something is missing, center on Austin and skip marker
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            ])
        ]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(str(breed_val)),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(str(name_val))
                ])
            ])
        ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server()


Dash app running on https://candleamerica-giantbetty-3000.codio.io/proxy/8050/
